In [1]:
# Trening model T5
# Za to uporablja datoteko PrevodiClarin.xlsx, ki je tudi na Git-u
# https://github.com/UL-FRI-NLP-Course-2022-23/nlp-course-parafrizerji/blob/main/database/PrevodiClarin.xlsx

In [2]:
!pip install datasets
!pip install transformers 
!pip install SentencePiece 
!pip install evaluate
!pip install rouge_score
!pip install tensorflow_text
!pip install torchtext


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# importi
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
import pyarrow as pa
import evaluate
from google.colab import drive
from datetime import datetime as dt
import os
from torch.utils.data import DataLoader

In [4]:
novModel = False
# Če je določena kot True, potem vzame model cjvt/t5-sl-small
# Če je False, pa pogleda na Google Drive in vzame shranjen model ter ga dodatno trenira

In [5]:
# povezava z drive in branje baze
drive.mount('/content/gdrive/', force_remount=True)
skupna = pd.read_excel("/content/gdrive/MyDrive/NLP/PrevodiClarin.xlsx")

Mounted at /content/gdrive/


In [6]:
# Na Google drive pogleda, kje se nahaja najnovejši model
# Model mora biti v direktoriju NLP
pot = "/content/gdrive/MyDrive/NLP"

potDoModela = ""
cas = 0

# Pregleda kateri model je zadnji
for filename in os.listdir(pot):
    if os.path.isdir(os.path.join(pot,filename)):
      if os.path.getmtime(os.path.join(pot,filename)) > cas:
        cas = os.path.getmtime(os.path.join(pot,filename))
        potDoModela = os.path.join(pot,filename)

print(potDoModela,cas)

/content/gdrive/MyDrive/NLP/2023-05-08T05:50:36.586395 1683525037.0


In [7]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("cjvt/t5-sl-small")

In [8]:
# Doda štiri kolone v Dataset

skupna['input_ids'] = None
skupna['attention_mask'] = None
skupna['labels'] = None
skupna['decoder_attention_mask'] = None

In [9]:
# Doda tokene v Dataset
for i in skupna.index:
    a = tokenizer("parafraza: "+skupna.at[i,'original'], padding="max_length", truncation=True, max_length=50)
    b = tokenizer(skupna.at[i,'parafraza'], padding="max_length", truncation=True, max_length=50)
    skupna.at[i,'input_ids'] = a.input_ids
    skupna.at[i,'attention_mask'] = a.attention_mask
    skupna.at[i,'labels'] = b.input_ids
    skupna.at[i,'decoder_attention_mask'] = b.attention_mask

In [10]:
# Zbriše nepotrebne kolone v Datasetu
skupna = skupna.drop(['original','anprevod','parafraza'],axis=1)

In [11]:
# Pretvorba in pandas dataframe v arrow dataset 
a = pa.Table.from_pandas(skupna)
skupni_dataset = Dataset(a)

In [12]:
# Tukaj izračuna koliko je 10% od vseh stavkov v bazi.
st_train = int(len(skupni_dataset)*0.7)
print(st_train)

18946


In [13]:
skupni_dataset.set_format("torch")

In [14]:
# izberem 70% za train in 30 za val
#moj_train_dataset = skupni_dataset.shuffle(seed=42).select(range(st_train))
#moj_val_dataset = skupni_dataset.shuffle(seed=42).select(range(len(skupni_dataset)-st_train))

#Lahko vzamem tudi fiksno število primerov
moj_train_dataset = skupni_dataset.shuffle(seed=42).select(range(1000))
moj_val_dataset = skupni_dataset.shuffle(seed=42).select(range(100))

In [15]:
print(moj_train_dataset)
print(moj_val_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
    num_rows: 1000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
    num_rows: 100
})


In [16]:
# Spremeni podatke v primerno obliko

train_dataloader = DataLoader(moj_train_dataset, shuffle=True, batch_size=64)
eval_dataloader = DataLoader(moj_val_dataset, batch_size=64)

In [17]:
from transformers import T5ForConditionalGeneration
if novModel:
  print("Prilagajam nov model.")
  model = T5ForConditionalGeneration.from_pretrained("cjvt/t5-sl-small")
else:
  print("Prilagajam obstoječ model")
  model = T5ForConditionalGeneration.from_pretrained(potDoModela)


Prilagajam obstoječ model


In [18]:
from torch.optim import AdamW
#optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer = AdamW(model.parameters(), lr=3e-4, eps=1e-8)

In [19]:
from transformers import get_scheduler

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [20]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32000, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32000, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [21]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    print(epoch)
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        #print(batch)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    metric = evaluate.load("bleu")
    model.eval()
    for batch in eval_dataloader:
      napovedi = []
      resnice = []
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
        outputs = model(**batch)

      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      for i in predictions:
        napovedi.append(tokenizer.decode(i.tolist()))
      predikcija = predictions[0].tolist()
      #predikcija = tf_sp.detokenize(predikcija)
      predikcija = [tokenizer.decode(predikcija)]
      zares = batch["labels"].tolist()
      for i in zares:
        resnice.append(tokenizer.decode(i))
      metric.add_batch(predictions=napovedi, references=resnice)

    rez = metric.compute()
    print(rez)

  0%|          | 0/16 [00:00<?, ?it/s]

0
{'bleu': 0.9872815897258838, 'precisions': [0.9944504117436448, 0.989342485549133, 0.9848705796573095, 0.9808682855040471], 'brevity_penalty': 0.9999104945178489, 'length_ratio': 0.9999104985232257, 'translation_length': 11172, 'reference_length': 11173}


In [22]:
# Shranjevanje modela na Google drive

potZaModel = "/content/gdrive/MyDrive/NLP/"+dt.now().isoformat()


model.save_pretrained(potZaModel, from_pt=True) 


In [23]:
Ta stavek je tu samo zato, da se izvajanje ustavi in preverjanje požene ročno

SyntaxError: ignored

In [24]:
stavkiZaPara = []
stavkiZaPara.append("Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.")
stavkiZaPara.append("Bila so čisto navadna torkova večerna poročila na POP TV, torej oddaja 24ur, in je Darja Zgonc čisto mirno, kot da napoveduje, da vreme pač bo, izrekla naslednjo napoved prispevka.")
stavkiZaPara.append("Življenje samohranilke z dvema otrokoma se vrti nevarno blizu roba.")
stavkiZaPara.append("Janez Vogrinc je dekan Pedagoške fakultete v Ljubljani. ")
stavkiZaPara.append("Je tudi vodja delovne skupine za pripravo novega nacionalnega programa vzgoje in izobraževanja za obdobje 2023–2033, ki jo je oblikovalo resorno ministrstvo.")
stavkiZaPara.append("Glasbeni megahit, ki ga je še danes mogoče slišati na številnih plesiščih (ali pa vsaj po radijskih valovih), je opeval tradicionalno moške lastnosti in njihove adute.")
stavkiZaPara.append("Upravno sodišče je sredi letošnjega januarja izdalo odločbo, v kateri je ugodilo tožbi kmeta, ki mu je bilo razlaščeno 17.785 kvadratnih metrov kmetijskega zemljišča v korist Občine Hoče - Slivnica.")
stavkiZaPara.append("V okviru 14. Dneva za spremembe Slovenska filantropija po vsej Sloveniji organizira akcije, ki spodbujajo državljane k aktivnemu odzivu na potrebe svoje skupnosti.")
stavkiZaPara.append("Slovenijo je zajela protestniška inflacija.")
stavkiZaPara.append("Protestov je čedalje več, njihova vrednost pa je čedalje manjša.")

In [25]:
for i in stavkiZaPara:
  #text =  "paraphrase: " + i + " </s>"
  #text =  i + " </s>"
  text =  "parafraza: " + i

  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.98,
      early_stopping=True,
      num_return_sequences=10
  )


  print ("\nOriginalni stavek:")
  print (i)
  print ("\n")
  print ("Parafrazirani stavki:")
  final_outputs =[]
  for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != i.lower() and sent not in final_outputs:
      final_outputs.append(sent)

  for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

  print("\n-----------------------\n")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(



Originalni stavek:
Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.


Parafrazirani stavki:
0: Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici ni zavarovanje.
1: Še nobena vlada ni odpravila dopolnilnega zdravstvenega zavarovanja, ki ni zavarovanje.
2: Še nobena vlada ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki pravzaprav ni zavarovanje.
3: Še nobena vlada ni mogla odpraviti dopolnilnega zdravstvenega zavarovanja, ki pravzaprav ni zavarovanje.
4: Še nobeno zdravstveno zavarovanje ni bilo vključeno v zavarovanje, ki je dejansko le zavarovanje.
5: Še nobena vlada ni mogla odpraviti dopolnilnega zdravstvenega zavarovanja, ki ni zavarovanje.
6: Še nobena vlada ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki ni zavarovanje.
7: Še nobena vlada ni mogla odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici ni zavarovanje.
8: Še nobena vlada ni posk